<h1>UST Reddit Projecy</h1></r>
<h2>Pipeline</h2>

In [ ]:
# gives accessibility to my goodle drive to fetch the data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h2>Generate dataset</h2>

In [ ]:
import pandas as pd
import numpy as np
import time
import re

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm

<h3>Read the raw data WITHOUT CLEANING</h3>

In [ ]:
fp = '/Users/victor/Desktop/CS/UST_summer/all_posts_clean_1.csv' # the directory of the raw data file, pipeliend from reddits

s_columns = ['id', 'author', 'created_utc', 'num_comments', 'clean_title', 'clean_selftext', 'num_title_emojis', 'num_text_emojis', 'title_length', 'text_length', 'score', 'num_replied_comments', 'num_replies_by_new', 'list_new_repliers', 'num_replies_by_old', 'list_old_repliers']

df = pd.read_csv(fp, low_memory=False, encoding='utf-8')[s_columns]

df.head() # test if it is read correctly

,id,author,created_utc,num_comments,clean_title,clean_selftext,num_title_emojis,num_text_emojis,title_length,text_length,score,num_replied_comments,num_replies_by_new,list_new_repliers,num_replies_by_old,list_old_repliers
0,mhj2yd,USWD_Bank,1.617235e+09,0.0,NFT Collectable USWD United States Web Dollar,removed,0.0,NaN,7.0,NaN,1.0,0.0,0.0,NaN,0.0,NaN
1,mhj3di,[deleted],1.617235e+09,1.0,Not use to Green Only use to Red,deleted,0.0,NaN,8.0,NaN,1.0,1.0,0.0,NaN,1.0,",VisualMod"
2,mhj3ew,KingMacias1,1.617235e+09,0.0,We made it to Vice News you dirty apes Any pub...,NaN,1.0,0.0,14.0,0.0,1.0,0.0,0.0,NaN,0.0,NaN
3,mhj3oa,jt989898,1.617235e+09,0.0,Polygon,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN,0.0,NaN
4,mhj3vq,KenAdamsonPrime,1.617235e+09,2.0,Ubiquiti UI gets PWND by hackers stock goes up,removed,0.0,NaN,9.0,NaN,1.0,1.0,0.0,NaN,2.0,",AutoModerator,VisualMod"


In [ ]:
# df = df[df.created_utc < 1590000000.0]

In [ ]:
start_time = time.time()
df.groupby('author').cumcount()
print("--- %s seconds ---" % (time.time() - start_time))

df["num_post"] = df.groupby('author').cumcount().to_numpy()
codes, uniques = pd.factorize(df["author"])
df["author_codes"] = codes

# df.to_csv('wsb_with_authorcode_num_post.csv')

--- 0.7724909782409668 seconds ---


<h2>for statistical tests ONLY</h2>

In [ ]:
indices = df.index[(df.created_utc >= 1609459200.0) & (df.created_utc <= 1612137599.0)].tolist()

# zeros = np.zeros(df.shape[0])
# for i in indices : zeros[i] = 1

# df['is_2021Jan'] = zeros


In [ ]:
number_of_replies = list()

index = df.index[(df['clean_selftext'] == 'deleted') | (df['clean_selftext'] == 'removed')].tolist()
for i in index:
    number_of_replies.append(df['num_comments'].iloc[i])
    
# len(number_of_replies)

nonzero_replies = np.count_nonzero(np.array(number_of_replies))
nonzero_replies

1059964

In [ ]:
print(df.shape)

(1553189, 19)


In [ ]:
# count the number of deleted and removed comments in each month from 2020-01-01 to 2021-05-01
date = datetime(2020,1,1,0)
time = []

for i in range(17):
    count_time = date + relativedelta(months=i)
    count_time = count_time.timestamp()
    time.append(count_time)

time


In [ ]:
def deleted_comments(left_bound, right_bound, dataframe):
    """_summary_

    Args:
        left_bound (_type_): _description_
        right_bound (_type_): _description_
        dataframe (_type_): _description_
    """
    indices = dataframe.index[(dataframe.created_utc >= left_bound) & (dataframe.created_utc <= right_bound)].tolist() # workout the total number of comments
    
    # print(indices)
    if len(indices) == 0:
        return 0
    
    unwanted_comments = 0
    for i in indices:
        if (dataframe['clean_selftext'].iloc[i] == 'deleted') | (dataframe['clean_selftext'].iloc[i] == 'removed'):
            unwanted_comments += 1
    
    return (unwanted_comments/len(indices))*100

In [ ]:
# workout the percentage of deleted commenets within the time range
unwanted_percent = []

for i in range (len(time)-1):
    left = time[i]
    right = time[i+1]
    
    
    unwanted_percent.append(deleted_comments(left,right,df))
    

In [ ]:
def replied_percent(left_bound, right_bound, dataframe):
    """_summary_

    Args:
        left_bound (_type_): _description_
        right_bound (_type_): _description_
        dataframe (_type_): _description_
    """
    indices = dataframe.index[(dataframe.created_utc >= left_bound) & (dataframe.created_utc <= right_bound)].tolist()
    
    negative_index = []
    negative_positive = 0
    
    for i in indices:
        if (dataframe['clean_selftext'].iloc[i] == 'deleted') | (dataframe['clean_selftext'].iloc[i] == 'removed'):
            negative_index.append(i)
    
    for i in negative_index:
        if dataframe.num_comments.iloc[i] > 0:
            negative_positive += 1
    
    negative_positive_rate = negative_positive / len(negative_index)
    
    positive_index = list(set(indices) - set(negative_index))
    positive_positive = 0
    
    for i in positive_index:
        if dataframe.num_comments.iloc[i] > 0:
            positive_positive += 1
    
    positive_positive_rate = positive_positive / len(positive_index)
    
    return (positive_positive_rate, negative_positive_rate)
        

In [ ]:
# workout the percentage of undeleted commenets got replied and deleted comments got replied within the time range
reply_percent = []

for i in range (len(time)-1):
    left = time[i]
    right = time[i+1]
    
    
    reply_percent.append(replied_percent(left,right,df))

In [ ]:
print(reply_percent)

In [ ]:
def AinB (A, B):
    check =  any(item in A for item in B)
    return check

def del_bot(name):
    tokens = re.split('[^a-zA-Z]', name)
    
    not_list = ['not', 'Not', 'NOt', 'NOT', 'nOt', 'nOT', 'noT', 'NoT']
    
    bot_list = ['bot', 'Bot', 'BOt', 'BOT', 'bOt', 'bOT', 'boT', 'BoT']
    
    if AinB(not_list, tokens) and AinB(bot_list, tokens):
        return False

    elif AinB(bot_list, tokens):
        return True
    
    return False

In [ ]:
# create a copy of the dataframe
botless_df = df.copy()

drop_indices = []

for i in range (len(botless_df['author'])):
    if del_bot(botless_df.author.iloc[i]):
        drop_indices.append(i)
drop_indices

In [ ]:
botless_df = botless_df.loc[~botless_df.index.isin(drop_indices)]

In [ ]:
# workout the percentage of deleted commenets within the time range
botless_unwanted_percent = []

for i in range (len(time)-1):
    left = time[i]
    right = time[i+1]
    
    
    botless_unwanted_percent.append(deleted_comments(left,right,botless_df))

In [ ]:
botless_unwanted_percent

In [ ]:
# workout the percentage of undeleted commenets got replied and deleted comments got replied within the time range
botless_reply_percent = []

for i in range (len(time)-1):
    left = time[i]
    right = time[i+1]
    
    
    botless_reply_percent.append(replied_percent(left,right,botless_df))

In [ ]:
botless_reply_percent

<h2>Change to use the new datasets</h2>
Utility function that helps to split the monthly dataset into daily ones

In [ ]:
# create a function that works out the timestamps of each day in a month
def workout_time(year, month):
  max_day = 31
  month_list = [4, 6, 9, 11]

  if year == 2020 and month == 2:
    max_day = 29
  elif month == 2:
    max_day = 28
  elif month in month_list:
    max_day = 30
  
  start_date = datetime(year, month, 1, 0) # always start at the first day on each month

  day_list = []

  for i in range(max_day):
    count_time = start_date + relativedelta(days=i)
    count_time = count_time.timestamp()
    day_list.append(count_time)
  
  return day_list

In [ ]:
# a funtion that creates sub_dataframes from the parent dataframe based on number of days
def create_subframe(day_list, parent_dataframe):
  df_list = []

  for i in range(len(day_list)-1):
    left = day_list[i]
    right = day_list[i+1]

    day_df = parent_dataframe[(parent_dataframe.created_utc >= left) & (parent_dataframe.created_utc < right)]
    
    df_list.append(day_df)
  
  return df_list


utility function to workout the centrality of the given dataframe

<h3>exmaine the monthly dataset</h3>

In [ ]:
# now we want to examine the 01-20 comments file
df_01_20 = pd.read_csv('/content/drive/MyDrive/UST_summer/01-20_comments.csv')

<h4>Clean the dataset first</h4>

In [ ]:
drop_indices = []

def remove_parent_bot(authors_dataframe, dataframe):
    j = authors_dataframe.columns.get_loc("parent_id")
    for i in range (len(authors_dataframe['parent_id'])):
        split_id = str(authors_dataframe.iloc[i, j]).split("_")[1]
        authorname = dataframe.loc[dataframe['id'] == split_id, 'author'].tolist()
        if authorname == []:
            continue
        else:
            if del_bot(authorname[0]):
                print('yes')
                drop_indices.append(str(authors_dataframe.iloc[i, j]))
    drop_indices

In [ ]:
def create_parent_children_df(dataframe):
    tqdm.pandas()
    level_1_comments = dataframe[dataframe['parent_id'] == dataframe['link_id']]

    sub_comments = dataframe[~dataframe.index.isin(level_1_comments.index)] #setting the sub comments being those with index that are not in level_1_comments

    posts_authors  = sub_comments.groupby('parent_id')['author', 'created_utc'].apply(list).reset_index(name='author')
    posts_authors['sub_comments'] = posts_authors['author'].progress_apply(lambda x:len(x))
    post_authors['created_utc'] 
    posts_authors.sort_values('sub_comments', inplace=True, ascending=False)

    return posts_authors

In [ ]:
def create_name_id_dict(dataframe):
    name_id = {}
    for i in range(dataframe.shape[0]):
        key = dataframe.id.iloc[i]
        value = dataframe.author.iloc[i]

        key_value_pair = {key:value}

        name_id.update(key_value_pair)
    return name_id

In [ ]:
def clean_by_parent(parent_children_dataframe, name_id_dicitonary):
    drop_indices = []
    parent_ids = parent_children_dataframe.parent_id
    indices = parent_children_dataframe.index
    for i in range(len(parent_ids)):
        uid = parent_ids[i].split('_')[-1]
        # use the dictionary to find the corresponding author name
        try:
            author_name = name_id_dicitonary[uid]
        except:
            continue
        if del_bot(author_name):
            drop_indices.append(indices[i])
    
    return drop_indices

In [ ]:
def clean_author(parent_children_dataframe):
    for i in range(parent_children_dataframe.shape[0]):
        bot = False
        df_index = parent_children_dataframe.index
        author_list = parent_children_dataframe.author.iloc[i]
        # print(author_list)
        for author in author_list:
            if del_bot(author):
                author_list.remove(author)
                bot = True
                # print(1)
        if bot:
            # print(1)
            parent_children_dataframe.loc[df_index[i]].at['author'] = author_list
            parent_children_dataframe.loc[df_index[i]].at['sub_comments'] = len(author_list)
    
    return parent_children_dataframe

-----------

In [ ]:
year, month = 2020, 1
day_list_01_20 = workout_time(year, month)

In [ ]:
df_list_01_20 = create_subframe(day_list_01_20, df_01_20)

In [ ]:
df_list_01_20

Create adjacency matrix:


In [ ]:
def get_all_authors(name_id_dict, child_parent_df):
    author_list = []
    j = child_parent_df.columns.get_loc("parent_id")
    for i in range(len(child_parent_df)):
        ids = str(child_parent_df.iloc[i, j]).split("_")[1]
        try:
            a_name = name_id_dict[ids]
            if a_name not in author_list:
                author_list.append(name_id_dict[ids])
        except:
            print('the parent post is created in a prior date')
            
        a_list = child_parent_df.author.iloc[i]
        for authors in a_list:
            if authors not in author_list:
                    author_list.append(authors)
    return author_list

In [ ]:
def get_all_authors(name_id_dict, dataframe):
    author_list = set()
    for i in range(dataframe.shape[0]):
        print(i)
        uid = dataframe.parent_id.iloc[i].split('_')[-1]

        try:
            author_list.add(name_id_dict[uid])
        except:
            print('the parent post is created in a prior date')

        author_list.update(tuple(dataframe.author.iloc[i]))
        
    return author_list

In [ ]:
def generate_adjacency_matrix(adj_matrix, p_c_df, name_dict):
    for i in adj_matrix.index:
        for k in range(p_c_df.shape[0]):
            author_list = p_c_df.author.iloc[k]
            if i in author_list:
                parent_name = p_c_df.parent_id.iloc[k].split("_")[1]
                try:
                    p_name = name_dict[parent_name]
                    print(p_name)
                    adj_matrix.loc[i, p_name] += 1
                    print(adj_matrix.loc[i, p_name])
                except:
                    continue
    return adj_matrix

In [ ]:
def generate_adj_matrix(author_list, dataframe, name_id_dict):
    adj_matrix = pd.DataFrame(0, index = author_list, columns = author_list)
    for i in range(dataframe.shape[0]):
        uid = dataframe.parent_id.iloc[i].split('_')[-1]
        
        try:
            parent_name = name_id_dict[uid]
        except:
            continue
        author_list = dataframe.author.iloc[i]

        for author in author_list:
            adj_matrix.loc[author, parent_name] += 1
    
    return adj_matrix

In [ ]:
a_matrix = pd.DataFrame(0, index = author_list, columns = author_list)

In [ ]:
a_matrix = generate_adjacency_matrix(a_matrix, posts_authors, name_id)

### **Formal Workflow**

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import networkx as nx
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
fp1 = "/Users/jerrylin/Downloads/01-20_comments.csv"
df_01 = pd.read_csv(fp1, low_memory = False, encoding = 'utf-8')  #get the Jan-2020 spreadsheet
df_01 = df_01[df_01.author != '[deleted]'] #remove authors that are deleted


In [ ]:
# create a function that works out the timestamps of each day in a month
def workout_time(year, month):
  max_day = 31
  month_list = [4, 6, 9, 11]

  if year == 2020 and month == 2:
    max_day = 29
  elif month == 2:
    max_day = 28
  elif month in month_list:
    max_day = 30
  
  start_date = datetime(year, month, 1, 0) # always start at the first day on each month

  day_list = []

  for i in range(max_day):
    count_time = start_date + relativedelta(days=i)
    count_time = count_time.timestamp()
    day_list.append(count_time)
  
  return day_list

# a funtion that creates sub_dataframes from the parent dataframe based on number of days
def create_subframe(day_list, parent_dataframe):
  df_list = []

  for i in range(len(day_list)-1):
    left = day_list[i]
    right = day_list[i+1]

    day_df = parent_dataframe[(parent_dataframe.created_utc >= left) & (parent_dataframe.created_utc < right)]
    
    df_list.append(day_df)
  
  return df_list

# a funtion that creates sub_dataframes from the parent dataframe based on number of days
def create_subframe(day_list, parent_dataframe):
  df_list = []

  for i in range(len(day_list)-1):
    left = day_list[i]
    right = day_list[i+1]

    day_df = parent_dataframe[(parent_dataframe.created_utc >= left) & (parent_dataframe.created_utc < right)]
    
    df_list.append(day_df)
  
  return df_list

In [ ]:
year, month = 2020, 1
day_list_01_20 = workout_time(year, month)
df_list_01_20 = create_subframe(day_list_01_20, df_01)   #creates day-by-day sub-dataframes

In [ ]:
#bot removal
def AinB (A, B):
    check =  any(item in A for item in B)
    return check

def del_bot(name):
    tokens = re.split('[^a-zA-Z]', name)
    
    not_list = ['not', 'Not', 'NOt', 'NOT', 'nOt', 'nOT', 'noT', 'NoT']
    
    bot_list = ['bot', 'Bot', 'BOt', 'BOT', 'bOt', 'bOT', 'boT', 'BoT']
    
    if AinB(not_list, tokens) and AinB(bot_list, tokens):
        return False

    elif AinB(bot_list, tokens):
        return True
    
    return False

In [ ]:
drop_indices = []


    drop_indices
    
def create_parent_children_df(dataframe):     #create the output dataframe
    tqdm.pandas()
    level_1_comments = dataframe[dataframe['parent_id'] == dataframe['link_id']]

    sub_comments = dataframe[~dataframe.index.isin(level_1_comments.index)] #setting the sub comments being those with index that are not in level_1_comments

    posts_authors  = sub_comments.groupby('parent_id')['author'].apply(list).reset_index(name='author')
    posts_authors['sub_comments'] = posts_authors['author'].progress_apply(lambda x:len(x))
    posts_authors.sort_values('sub_comments', inplace=True, ascending=False)

    return posts_authors

def create_name_id_dict(dataframe):    #create id:author dataframe
    name_id = {}
    for i in range(dataframe.shape[0]):
        key = dataframe.id.iloc[i]
        value = dataframe.author.iloc[i]

        key_value_pair = {key:value}

        name_id.update(key_value_pair)
    return name_id

def clean_by_parent(parent_children_dataframe, name_id_dicitonary):   #clean bots from parent_id
    drop_indices = []
    parent_ids = parent_children_dataframe.parent_id
    indices = parent_children_dataframe.index
    for i in range(len(parent_ids)):
        id = parent_ids[i].split('_')[-1]
        # use the dictionary to find the corresponding author name
        try:
            author_name = name_id_dicitonary[id]
        except:
            continue
        if del_bot(author_name):
            drop_indices.append(indices[i])
    
    return drop_indices

def clean_author(parent_children_dataframe):    #clean bots from authors
    for i in range(parent_children_dataframe.shape[0]):
        bot = False
        df_index = parent_children_dataframe.index
        author_list = parent_children_dataframe.author.iloc[i]
        # print(author_list)
        for author in author_list:
            if del_bot(author):
                author_list.remove(author)
                bot = True
                # print(1)
        if bot:
            # print(1)
            parent_children_dataframe.loc[df_index[i]].at['author'] = author_list
            parent_children_dataframe.loc[df_index[i]].at['sub_comments'] = len(author_list)
    
    return parent_children_dataframe

In [ ]:
day_df = df_list_01_20[1]
posts_authors = create_parent_children_df(day_df)
name_id = create_name_id_dict(day_df)
drop_indices = clean_by_parent(posts_authors, name_id)
drop_indices
posts_authors = posts_authors.drop(drop_indices)
posts_authors = clean_author(posts_authors)
posts_authors

In [ ]:
def get_all_authors(name_id_dict, dataframe):   #get all author names
    author_list = set()
    for i in range(dataframe.shape[0]):
        uid = dataframe.parent_id.iloc[i].split('_')[-1]

        try:
            author_list.add(name_id_dict[uid])
        except:
            continue

        author_list.update(tuple(dataframe.author.iloc[i]))
        
    return author_list

def generate_adj_matrix(author_list, dataframe, name_id_dict):   #generate the adjacency matrix
    adj_matrix = pd.DataFrame(0, index = author_list, columns = author_list)
    for i in range(dataframe.shape[0]):
        uid = dataframe.parent_id.iloc[i].split('_')[-1]
        
        try:
            parent_name = name_id_dict[uid]
        except:
            continue
        author_list = dataframe.author.iloc[i]

        for author in author_list:
            adj_matrix.loc[author, parent_name] += 1
    
    return adj_matrix

In [ ]:
author_list = get_all_authors(name_id, posts_authors)
generate_adj_matrix(author_list, posts_authors, name_id)
adj_matrix.to_csv('file name')